## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import sys
sys.path.append("..")
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hofn,IS,64.2539,-15.2082,53.80,83,34,13.44,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,45.39,85,88,4.45,overcast clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,53.60,87,90,12.66,overcast clouds
3,3,Ketchikan,US,55.3422,-131.6461,61.27,77,90,6.91,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,71.78,61,100,26.08,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Jamestown,US,42.0970,-79.2353,80.29,67,1,4.00,clear sky
7,7,Bethel,US,41.3712,-73.4140,85.19,80,75,1.01,broken clouds
16,16,Morehead,US,37.2711,-87.1764,80.35,80,1,3.44,clear sky
26,26,Buchanan,LR,5.8808,-10.0467,76.96,83,90,10.76,overcast clouds
27,27,Salalah,OM,17.0151,54.0924,77.09,94,90,5.75,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
660,660,Basco,PH,20.4487,121.9702,84.31,73,100,4.81,overcast clouds
661,661,Ayna,PE,-12.6500,-73.9167,77.50,67,77,5.08,broken clouds
662,662,Male,MV,4.1748,73.5089,84.15,84,40,6.91,scattered clouds
663,663,Padang,ID,-0.9492,100.3543,76.91,87,77,2.37,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Jamestown,US,80.29,clear sky,42.0970,-79.2353,
7,Bethel,US,85.19,broken clouds,41.3712,-73.4140,
16,Morehead,US,80.35,clear sky,37.2711,-87.1764,
26,Buchanan,LR,76.96,overcast clouds,5.8808,-10.0467,
27,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
28,Sabha,LY,88.86,clear sky,27.0377,14.4283,
29,Yumen,CN,77.81,few clouds,40.2833,97.2000,
30,The Valley,AI,87.94,scattered clouds,18.2170,-63.0578,
31,San Quintin,MX,78.10,clear sky,30.4833,-115.9500,
33,Kapaa,US,80.58,light rain,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df.replace('',np.nan, inplace=True)
clean_hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Jamestown,US,80.29,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Bethel,US,85.19,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
16,Morehead,US,80.35,clear sky,37.2711,-87.1764,CCI Express Inn
26,Buchanan,LR,76.96,overcast clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
27,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
...,...,...,...,...,...,...,...
658,Meulaboh,ID,76.03,overcast clouds,4.1363,96.1285,Kost Pria
660,Basco,PH,84.31,overcast clouds,20.4487,121.9702,Nathaniel's Lodge
662,Male,MV,84.15,scattered clouds,4.1748,73.5089,Hotel Octave Maldives
663,Padang,ID,76.91,broken clouds,-0.9492,100.3543,Mercure Padang


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))